<a href="https://colab.research.google.com/github/AymenTlili131/The-sound-of-Ai-Hack-V1.0/blob/working_proposals/MIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
from os import walk
import librosa.display
from natsort import natsorted
import pickle
from tqdm.notebook import tqdm

In [2]:
from google.colab import drive
###working with a Drive version of the dataset
drive.mount('/content/drive',force_remount= True)

Mounted at /content/drive


In [13]:
def extract_features(x, 
                    #  feature, 
                     win_length,
                     n_mfcc, 
                     sr, 
                     n_mels):
    #MIR extraction 
    hop_length = int(win_length/2)
    rms = librosa.feature.rms(x, frame_length=win_length, hop_length=hop_length, center=True)[0]
    spec_cent = librosa.feature.spectral_centroid(y=x ,sr=sr, hop_length=hop_length, win_length=win_length)[0]  
    zcr = librosa.feature.zero_crossing_rate(x, frame_length=win_length, hop_length=hop_length)[0]
    #D = np.abs(librosa.stft(x))**2
    #mel = librosa.feature.melspectrogram(S=D, sr=sr) 
    mel = librosa.feature.melspectrogram(x, sr= sr, hop_length= hop_length,n_mels= n_mels)
    mfcc = librosa.feature.mfcc(x, sr= sr,n_mfcc= n_mfcc)
    duration = x.shape[0]/sr
    bpm = librosa.beat.tempo(x, sr= sr)[0]

    features = {'rms': rms,
                'mfcc': mfcc,
                'mel': mel,
                'sc': spec_cent,
                'zcr': zcr,
                'duration': duration,
                'bpm': bpm
                }

    return features

In [ ]:
#####################
#####################
#####################
#####################
#####################
#####################
#####################

In [3]:
artist= os.listdir('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/')
print(artist)

['Sweden', 'Mexico', 'USA', 'India', 'MIR_india.csv', 'MIR_mexico.csv', 'MIR_usa.csv', 'MIR_Sweden.csv']


In [33]:
from pathlib import Path

def parse_dataset_wavs(dataset_path: str, verbose: bool=False) -> tuple():
    dirs_wavs = sorted(Path(dataset_path).glob('**/*.wav'))
    # print(len(dirs_wavs))
    artists = [str(x).split('/')[-3] for x in dirs_wavs]
    songs = [str(x).split('/')[-1] for x in dirs_wavs]
    audio_paths = [str(dirs_wavs[idx]) for idx, dir in enumerate(dirs_wavs)]
    if verbose:
        for i in range(len(dirs_wavs)):
            print(f'Artist {i}',artists[i])
            print(f'Song {i}', songs[i])
            print(f'Audio {i}', audio_paths[i])
            print('\n')
    assert len(dirs_wavs) == len(artists) == len(songs) == len(audio_paths)

    return artists, songs, audio_paths

def create_waveforms(audio_path: list, sr: int=44100) -> list:
    audio_files = []
    for audio in audio_path:
        x, fs = librosa.load(audio, sr=sr)
        audio_files.append(x)
    audio_files = np.array(audio_files)

    return audio_files

In [7]:
import IPython.display as ipd
ipd.Audio(audio_files[0][:441000],rate=44100)


In [32]:
def create_dataframe(dataset_path, display_dataframe: bool=True, sr=44100, win_length=2048, duration=30.0, n_mfcc=13, n_mels=128):

    artists, songs, audio_paths = parse_dataset_wavs(dataset_path)
    audio_files = create_waveforms(audio_paths, sr)

    MEL, RMS, SC, ZCR, DUR, BPM = [], [], [], [], [], []
    for i in tqdm(audio_files, desc='Extracting features'):
        features_dict = extract_features(i[:int(sr * duration)], win_length=win_length, n_mfcc=n_mfcc, sr=sr, n_mels=n_mels)
        MEL.append(features_dict['mel'])
        RMS.append(np.array(features_dict['rms']))
        SC.append(np.array(features_dict['sc']))
        ZCR.append(np.array(features_dict['zcr']))
        DUR.append(features_dict['duration'])
        BPM.append(features_dict['bpm'])

    dict_info = {
    "artist": artists,
    "song": songs,
    "mel": MEL,
    "rms(mean)": np.mean(RMS, axis=1),
    "rms(var)": np.var(RMS, axis=1),
    "sc(mean)": np.mean(SC, axis=1),
    "sc(var)": np.var(SC, axis=1),
    "zcr(mean)": np.mean(ZCR, axis=1),
    "zcr(var)": np.var(ZCR, axis=1),
    "duration": DUR,
    "bpm": BPM,
    }

    audio_features = pd.DataFrame(my_dict)
    if display_dataframe:
        display(audio_features)
    
    return audio_features

In [37]:
# DATASET_PATH = '/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/'
# dirs = os.listdir(DATASET_PATH)
# print(dirs)

mir_sweden = create_dataframe('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/Sweden/')
mir_india = create_dataframe('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/India/')
mir_usa = create_dataframe('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/USA/')
mir_mexico = create_dataframe('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/Mexico/')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


  0%|          | 0/49 [00:00<?, ?it/s]

,artist,song,mel,rms(mean),rms(var),sc(mean),sc(var),zcr(mean),zcr(var),duration,bpm
0,23,01 Bla bla (Intro).wav,"[[9.290754e-18, 1.9965177e-07, 5.3759522e-05, ...",0.145916,0.008809,2293.129278,1.184585e+06,0.052493,0.001241,30.0,135.999178
1,"Alesso, Zara Larsson",01 Words (feat. Zara Larsson).wav,"[[0.0, 6.4441608e-09, 1.4698058e-05, 1.5179209...",0.159152,0.003780,2620.256999,2.234224e+06,0.058505,0.002622,30.0,126.048018
2,Ant Wan,01 Komplicerat.wav,"[[0.0, 0.0016549706, 0.009627789, 0.015177699,...",0.236703,0.011463,2319.940081,2.584504e+06,0.045590,0.001675,30.0,132.512019
3,BELL,01 MISSFÖRSTÅTT.wav,"[[0.0, 0.0, 0.0, 0.0011059896, 0.12315781, 0.1...",0.235809,0.011439,1961.185749,1.484623e+06,0.039298,0.001049,30.0,109.956782
4,Babblarna,01 Babblarnas vaggvisa.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.077918,0.003925,1233.312531,1.172904e+06,0.024619,0.000440,30.0,92.285156
5,Benson Boone,01 In The Stars.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.097384,0.007683,2327.358398,2.277402e+06,0.050736,0.003789,30.0,143.554688
6,Beyoncé,01 BREAK MY SOUL.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.191381,0.028487,1851.212532,1.755613e+06,0.038733,0.001632,30.0,114.843750
7,Bolaget,01 Kan Inte Gå.wav,"[[1.86534e-06, 7.6159574e-07, 2.9103168e-07, 1...",0.098779,0.001374,2880.289286,2.981776e+06,0.057042,0.004176,30.0,139.674831
8,Cornelia Jakobs,01 Hold Me Closer.wav,"[[0.0, 1.6144596e-15, 1.0312249e-08, 2.7211337...",0.178832,0.007733,2330.989644,1.370606e+06,0.041851,0.001528,30.0,120.185320
9,Dani M,01 Vidare.wav,"[[4.966081e-07, 0.07530498, 0.2711832, 0.00874...",0.106586,0.015931,2110.966626,1.154765e+06,0.038104,0.000784,30.0,97.508844


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


  0%|          | 0/49 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

,artist,song,mel,rms(mean),rms(var),sc(mean),sc(var),zcr(mean),zcr(var),duration,bpm
0,23,01 Bla bla (Intro).wav,"[[9.290754e-18, 1.9965177e-07, 5.3759522e-05, ...",0.145916,0.008809,2293.129278,1.184585e+06,0.052493,0.001241,30.0,135.999178
1,"Alesso, Zara Larsson",01 Words (feat. Zara Larsson).wav,"[[0.0, 6.4441608e-09, 1.4698058e-05, 1.5179209...",0.159152,0.003780,2620.256999,2.234224e+06,0.058505,0.002622,30.0,126.048018
2,Ant Wan,01 Komplicerat.wav,"[[0.0, 0.0016549706, 0.009627789, 0.015177699,...",0.236703,0.011463,2319.940081,2.584504e+06,0.045590,0.001675,30.0,132.512019
3,BELL,01 MISSFÖRSTÅTT.wav,"[[0.0, 0.0, 0.0, 0.0011059896, 0.12315781, 0.1...",0.235809,0.011439,1961.185749,1.484623e+06,0.039298,0.001049,30.0,109.956782
4,Babblarna,01 Babblarnas vaggvisa.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.077918,0.003925,1233.312531,1.172904e+06,0.024619,0.000440,30.0,92.285156
5,Benson Boone,01 In The Stars.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.097384,0.007683,2327.358398,2.277402e+06,0.050736,0.003789,30.0,143.554688
6,Beyoncé,01 BREAK MY SOUL.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.191381,0.028487,1851.212532,1.755613e+06,0.038733,0.001632,30.0,114.843750
7,Bolaget,01 Kan Inte Gå.wav,"[[1.86534e-06, 7.6159574e-07, 2.9103168e-07, 1...",0.098779,0.001374,2880.289286,2.981776e+06,0.057042,0.004176,30.0,139.674831
8,Cornelia Jakobs,01 Hold Me Closer.wav,"[[0.0, 1.6144596e-15, 1.0312249e-08, 2.7211337...",0.178832,0.007733,2330.989644,1.370606e+06,0.041851,0.001528,30.0,120.185320
9,Dani M,01 Vidare.wav,"[[4.966081e-07, 0.07530498, 0.2711832, 0.00874...",0.106586,0.015931,2110.966626,1.154765e+06,0.038104,0.000784,30.0,97.508844


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


  0%|          | 0/50 [00:00<?, ?it/s]

,artist,song,mel,rms(mean),rms(var),sc(mean),sc(var),zcr(mean),zcr(var),duration,bpm
0,23,01 Bla bla (Intro).wav,"[[9.290754e-18, 1.9965177e-07, 5.3759522e-05, ...",0.145916,0.008809,2293.129278,1.184585e+06,0.052493,0.001241,30.0,135.999178
1,"Alesso, Zara Larsson",01 Words (feat. Zara Larsson).wav,"[[0.0, 6.4441608e-09, 1.4698058e-05, 1.5179209...",0.159152,0.003780,2620.256999,2.234224e+06,0.058505,0.002622,30.0,126.048018
2,Ant Wan,01 Komplicerat.wav,"[[0.0, 0.0016549706, 0.009627789, 0.015177699,...",0.236703,0.011463,2319.940081,2.584504e+06,0.045590,0.001675,30.0,132.512019
3,BELL,01 MISSFÖRSTÅTT.wav,"[[0.0, 0.0, 0.0, 0.0011059896, 0.12315781, 0.1...",0.235809,0.011439,1961.185749,1.484623e+06,0.039298,0.001049,30.0,109.956782
4,Babblarna,01 Babblarnas vaggvisa.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.077918,0.003925,1233.312531,1.172904e+06,0.024619,0.000440,30.0,92.285156
5,Benson Boone,01 In The Stars.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.097384,0.007683,2327.358398,2.277402e+06,0.050736,0.003789,30.0,143.554688
6,Beyoncé,01 BREAK MY SOUL.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.191381,0.028487,1851.212532,1.755613e+06,0.038733,0.001632,30.0,114.843750
7,Bolaget,01 Kan Inte Gå.wav,"[[1.86534e-06, 7.6159574e-07, 2.9103168e-07, 1...",0.098779,0.001374,2880.289286,2.981776e+06,0.057042,0.004176,30.0,139.674831
8,Cornelia Jakobs,01 Hold Me Closer.wav,"[[0.0, 1.6144596e-15, 1.0312249e-08, 2.7211337...",0.178832,0.007733,2330.989644,1.370606e+06,0.041851,0.001528,30.0,120.185320
9,Dani M,01 Vidare.wav,"[[4.966081e-07, 0.07530498, 0.2711832, 0.00874...",0.106586,0.015931,2110.966626,1.154765e+06,0.038104,0.000784,30.0,97.508844


In [38]:
from google.colab import files

DATASET_PATH = '/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/'

mir_sweden.to_csv(os.path.join(DATASET_PATH, 'MIR_Sweden.csv'), encoding='utf-8-sig', index=False)
mir_mexico.to_csv(os.path.join(DATASET_PATH, 'MIR_Mexico.csv'), encoding='utf-8-sig', index=False)
mir_usa.to_csv(os.path.join(DATASET_PATH, 'MIR_USA.csv'), encoding='utf-8-sig', index=False)
mir_india.to_csv(os.path.join(DATASET_PATH, 'MIR_India.csv'), encoding='utf-8-sig', index=False)